In [1]:
from TSPGraph import Graph
import numpy as np
from functools import lru_cache
import copy
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
def random_solution():
    sol = np.random.choice(np.arange(200),size=100,replace=False)
    return np.append(sol,sol[0])

In [3]:
combination_pairs = [(x,y) for x in range(1,100) for y in range(x+2, 100)]
combination_pairs_edges = [(x,y) for x in range(1,100) for y in range(x+2, 100)]

In [4]:
def ExchangeVerticesDelta(graph, solution, newNode, i):
    prevVertex = solution[i - 1]
    nextVertex = solution[(i + 1) % len(solution)]

    costOut = graph.dist_matrix[prevVertex][solution[i]] + graph.dist_matrix[solution[i]][nextVertex]
    
    costIn = graph.dist_matrix[prevVertex][newNode] + graph.dist_matrix[newNode][nextVertex]

    return costIn - costOut

def exchangeVerticesDeltaNoSwap(graph, solution, newNode, i):
    prevVertex = solution[i - 1]
    nextVertex = solution[(i + 1) % len(solution)]

    costOut = graph.dist_matrix[prevVertex][solution[i]] + graph.dist_matrix[solution[i]][nextVertex]

    incomingVertex = solution[(i + 2) % len(solution)]
    costOut += graph.dist_matrix[nextVertex][incomingVertex]
    costIn = graph.dist_matrix[prevVertex][nextVertex] + graph.dist_matrix[nextVertex][newNode] + (
        graph.dist_matrix[newNode][incomingVertex]
    )

    return costIn - costOut

In [5]:
def ExchangeEdges(solution, i, j):
    result = copy.deepcopy(solution)
    if i < j:
        result = np.concatenate((result[:i], result[i:j+1][::-1], result[j+1:]))
    else:
        result = np.concatenate((result[i:][::-1], result[j+1:i], result[:j+1][::-1]))
    return result

def ExchangeEdgesDelta(graph, solution, i, j):
    prevVertex = solution[i - 1]
    nextVertex = solution[(j + 1) % len(solution)]
    
    costOut = graph.dist_matrix[solution[i]][prevVertex] + graph.dist_matrix[solution[j]][nextVertex]
    costIn = graph.dist_matrix[solution[j]][prevVertex] + graph.dist_matrix[solution[i]][nextVertex]
    
    return costIn - costOut

In [6]:
def insertNode(solution, newNode, i):
  result = copy.deepcopy(solution)
  result[i] = newNode

  return result

def insertNodeNoSwap(solution, newNode, insertID):
  result = copy.deepcopy(solution)
  
  result[insertID] = result[insertID + 1]
  result[insertID + 1] = newNode

  return result

In [7]:
def getCandidateNeighbourhood(graph, solution, candidates):
    neighbours = []
    for node1Index, node1 in enumerate(solution):
        nodeCandidates = candidates[node1]
        nextIndex = (node1Index + 1) % len(solution)
        previousIndex = node1Index - 1
        
        for node2 in nodeCandidates:
            indicesNode2 = np.where(solution == node2)[0]
            isNode2InSolution = len(indicesNode2) > 0

            if isNode2InSolution:
                node2Index = indicesNode2[0]

                if not (nextIndex >= node2Index or node1Index >= node2Index - 1):   
                    neighbours.append((ExchangeEdgesDelta(graph, solution, nextIndex, node2Index), ExchangeEdges(solution, nextIndex, node2Index)))
                    neighbours.append((ExchangeEdgesDelta(graph, solution, node1Index, node2Index - 1), ExchangeEdges(solution, node1Index, node2Index - 1)))
            else:
                neighbours.append((ExchangeVerticesDelta(graph, solution, node2, nextIndex), insertNode(solution, node2, nextIndex)))
                neighbours.append((exchangeVerticesDeltaNoSwap(graph, solution, node2, previousIndex), insertNodeNoSwap(solution, node2, previousIndex)))
    
    return neighbours

In [8]:
def SteepestLocalSearch(graph, initial_solution, candidates):
    current_solution = initial_solution
    neighbourhood = getCandidateNeighbourhood(graph, current_solution, candidates)
    bestDelta = 0
    while(len(neighbourhood)):
        deltas = np.array([delta for delta, _ in neighbourhood])
        best_index = np.argmin(deltas)
        bestDelta, current_solution = neighbourhood[best_index]

        if bestDelta >= 0:
            break

        neighbourhood = getCandidateNeighbourhood(graph, current_solution, candidates)
    return graph.cycle_cost(current_solution), current_solution

In [9]:
random_solutions = [random_solution() for i in range(200)]
previous_solutions_D = np.load('Data/WeightedRegretResults/d_weighted_regret_all.npy')
previous_solutions_C = np.load('Data/WeightedRegretResults/c_weighted_regret_all.npy')
previous_solutions_B = np.load('Data/WeightedRegretResults/b_weighted_regret_all.npy')
previous_solutions_A = np.load('Data/WeightedRegretResults/a_weighted_regret_all.npy')

In [10]:
def SteepestExperiment(graph, init_solutions):
    steep_random_edge = []
    cycles = []
    startingCandidates = getCandidates(graph)
    for i in tqdm(range(200)):
        cost, cycle = SteepestLocalSearch(graph, init_solutions[i], startingCandidates)
        steep_random_edge.append(cost)
        cycles.append(cycle)
    return np.array(steep_random_edge), np.array(cycles)

In [11]:
def getCandidates(graph):
  return graph.dist_matrix.argsort(-1)[:,:10]

In [12]:
def printResults(x):
  print("min: {} max: {} avg: {}".format(np.min(x), np.max(x), np.mean(x)))

In [14]:
resEdgeD = SteepestExperiment(Graph('D'), random_solutions)
print('steepest random edge candidates')
printResults(resEdgeD)

100%|██████████| 200/200 [01:47<00:00,  1.85it/s]

steepest random edge candidates
min: 59974 max: 70926 avg: 64797.455


In [15]:
resEdgeC = SteepestExperiment(Graph('C'), random_solutions)
print('steepest random edge candidates')
printResults(resEdgeC)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:49<00:00,  1.83it/s]

steepest random edge candidates
min: 63479 max: 74803 avg: 68842.895
